# Tabula Data Extraction API

In [1]:
import pandas as pd
import numpy as np
import spacy
import re

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
#https://www.cnbc.com/2019/05/03/hsbc-reports-2019-first-quarter-earnings.html
txt = """\
The bank said its reported profit before tax in the first quarter \
was $6.213 billion, a 30.7% jump from last year’s $4.755 billion. \
Analyst forecasts compiled by Refinitiv showed that the \
bank’s reported profit before tax was expected to come in \
at $5.399 billion for the January to March period. \
"""
doc = nlp(txt)

In [518]:
txt = """\
For example, U.S. imports from China almost doubled within five years \
from $51.5 billion ($84.2 billion in 2019 dollars) in 1996 to $102 billion \
($148 billion in 2019 dollars) in 2001.\
"""
doc = nlp(txt)

In [69]:
txt ="""\
I really like the new design of your website! \
The new design is awful! \
"""
doc = nlp(txt)

In [4]:
txt = """\
In 2003 GDP growth is 20%. In 2004 GDP growth 30%. In 2005 it is 40%.
"""
doc = nlp(txt)

In [33]:
txt = """\
Apple has a market share of 10.1%, \
Samsung has 22%, \
Huawei has 17.6%, \
Xiaomi has 9.7%, \
OPPO has 8.9%, \
and others have 31% \
"""
doc = nlp(txt)

In [70]:
for ent in doc:
    print((ent.text, ent.sentiment))

('I', 0.0)
('really', 0.0)
('like', 0.0)
('the', 0.0)
('new', 0.0)
('design', 0.0)
('of', 0.0)
('your', 0.0)
('website', 0.0)
('!', 0.0)
('The', 0.0)
('new', 0.0)
('design', 0.0)
('is', 0.0)
('awful', 0.0)
('!', 0.0)


In [46]:
from spacy import displacy

In [47]:
#displacy.serve(doc, style="dep")

/opt/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [51]:
ns = list(doc.noun_chunks)
tmp_ns_with_ancestors = [(n, self_with_ancestor(n.root)) for n in doc.noun_chunks]
ns_apple = tmp_ns_with_ancestors[0]
ns_samsung = tmp_ns_with_ancestors[3]

tmp_ents_with_ancestors = sorted([(ent, self_with_ancestor(ent.root))
                                  for ent in doc.ents],
                                 key=lambda x: x[0].start_char)
per22 = tmp_ents_with_ancestors[3]

In [50]:
tmp_ns_with_ancestors

[(Apple, [Apple, has, has, has, has, has]),
 (a market share, [share, has, has, has, has, has]),
 (10.1%, [%, of, share, has, has, has, has, has]),
 (Samsung, [Samsung, has, has, has, has]),
 (22%, [%, has, has, has, has]),
 (Huawei, [Huawei, has, has, has]),
 (17.6%, [%, has, has, has]),
 (Xiaomi, [Xiaomi, has, has]),
 (9.7%, [%, has, has]),
 (OPPO, [OPPO, has]),
 (8.9%, [%, has]),
 (others, [others, have, has]),
 (31%, [%, have, has])]

In [49]:
tmp_ents_with_ancestors

[(Apple, [Apple, has, has, has, has, has]),
 (10.1%, [%, of, share, has, has, has, has, has]),
 (Samsung, [Samsung, has, has, has, has]),
 (22%, [%, has, has, has, has]),
 (Huawei, [Huawei, has, has, has]),
 (17.6%, [%, has, has, has]),
 (Xiaomi, [Xiaomi, has, has]),
 (9.7%, [%, has, has]),
 (OPPO, [OPPO, has]),
 (8.9%, [%, has]),
 (31%, [%, have, has])]

In [55]:
ns_samsung

(Samsung, [Samsung, has, has, has, has])

In [56]:
per22

(22%, [%, has, has, has, has])

In [58]:
ns_samsung[1][1:] == per22[1][1:]

True

In [59]:
n_common_tail(ns_samsung[1], per22[1])

4

In [60]:
n_common_tail(ns_apple[1], per22[1])

4

In [64]:
for ent in doc.ents:
    print((ent.text, ent.start_char, ent.end_char, ent.label_, ent.sentiment))

In [71]:
def self_with_ancestor(token):
    return [token] + list(token.ancestors)

def n_common_tail(lst1, lst2):
    L = min(len(lst1), len(lst2))
    for i in range(1, L+1):
        if lst1[-i] != lst2[-i]:
            return i-1
    return L

def closest_entry_in_tree(token_ancestor, other_tokens_ancestors, default=None):
    # token_ancestor is (token, ancestor_list)
    # other_tokens_ancestors is list of (token, ancestor_list)
    if not other_tokens_ancestors:
        return default
    tok_ancestors = list(token_ancestor[1])
    # restrict to be in the same tree, i.e. at least one common ancestor
    n_commons = [(x, n_common_tail(tok_ancestors, x[1])) for x in other_tokens_ancestors]
    n_commons_in_tree = [x for x in n_commons if x[1] > 0]
    if not n_commons_in_tree:
        return default
    closest = max(n_commons_in_tree, key=lambda x: x[1])
    return closest[0]

def closest_token_in_tree(token_ancestor, other_tokens_ancestors, default=None):
    entry = closest_entry_in_tree(token_ancestor, other_tokens_ancestors, default)
    if entry:
        return entry[0]
    return entry

def tabular_data_extraction_1(doc):
    ents_with_ancestors = sorted([(ent, self_with_ancestor(ent.root))
                                  for ent in doc.ents],
                                 key=lambda x: x[0].start_char)
    values_with_ancestors = [x for x in ents_with_ancestors if x[0].label_ in ['MONEY', 'PERCENT', 'CARDINAL', 'QUANTITY']]
    dates_with_ancestors = [x for x in ents_with_ancestors if x[0].label_ in ['DATE', 'TIME']]
    
    # sort by increasing length of ancestor list, so that in case of tie, a shorter tail is preferred
    ns_with_ancestors = sorted([(n, self_with_ancestor(n.root)) for n in doc.noun_chunks],
                               key=lambda x: len(x[1]))
    ns_subj = [n for n in ns_with_ancestors if n[0].root.dep_ == 'nsubj']
    return([(x[0],
             closest_token_in_tree(x, dates_with_ancestors),
             closest_token_in_tree(x, ns_subj))
            for x in values_with_ancestors])

In [72]:
tabular_data_extraction_1(doc)

[]

In [73]:
def test_tabular_data_extraction(text):
    doc = nlp(text)
    print('# entities')
    for ent in doc.ents:
        print((ent.text, ent.start_char, ent.end_char, ent.label_, list(ent.root.ancestors)))
    print('# noun chunks')
    for chunk in doc.noun_chunks:
        print((chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text))
        
        print('== ')
    print('#')
    return tabular_data_extraction_1(doc)

In [74]:
test_tabular_data_extraction('In 2019, Apple has a market share of 10.1%, \
Samsung has 22%, \
Huawei has 17.6%, \
Xiaomi has 9.7%, \
OPPO has 8.9%, \
and others have 31% \
')

# entities
('2019', 3, 7, 'DATE', [In, has, has, has, has, has])
('Apple', 9, 14, 'ORG', [has, has, has, has, has])
('10.1%', 37, 42, 'PERCENT', [of, share, has, has, has, has, has])
('Samsung', 44, 51, 'ORG', [has, has, has, has])
('22%', 56, 59, 'PERCENT', [has, has, has, has])
('Huawei', 61, 67, 'ORG', [has, has, has])
('17.6%', 72, 77, 'PERCENT', [has, has, has])
('Xiaomi', 79, 85, 'ORG', [has, has])
('9.7%', 90, 94, 'PERCENT', [has, has])
('OPPO', 96, 100, 'ORG', [has])
('8.9%', 105, 109, 'PERCENT', [has])
('31%', 127, 130, 'PERCENT', [have, has])
# noun chunks
('Apple', 'Apple', 'nsubj', 'has')
== 
('a market share', 'share', 'dobj', 'has')
== 
('10.1%', '%', 'pobj', 'of')
== 
('Samsung', 'Samsung', 'nsubj', 'has')
== 
('22%', '%', 'dobj', 'has')
== 
('Huawei', 'Huawei', 'nsubj', 'has')
== 
('17.6%', '%', 'dobj', 'has')
== 
('Xiaomi', 'Xiaomi', 'nsubj', 'has')
== 
('9.7%', '%', 'dobj', 'has')
== 
('OPPO', 'OPPO', 'nsubj', 'has')
== 
('8.9%', '%', 'dobj', 'has')
== 
('others', 'ot

[(10.1%, 2019, Apple),
 (22%, 2019, Samsung),
 (17.6%, 2019, Huawei),
 (9.7%, 2019, Xiaomi),
 (8.9%, 2019, OPPO),
 (31%, 2019, others)]

In [6]:
def get_date(dates, sent, ent_order):
    if (len(dates) == 0):
        ent_date = []
    else:
        ent_date = [d['value'] for d in dates if d['sent'] == sent]
        if (len(ent_date) == 0):
            ent_date = [d['value'] for d in dates if d['order'] < ent_order]
    return([ent_date[0] if len(ent_date) != 0 else ''][0])

def tabular_data_extraction(doc):
    sents = [i for l in [str(s).split(',') for s in list(doc.sents)] for i in l]
    sents_start = np.array([txt.find(sents[i]) for i in range(len(sents))])
    sents_end = sents_start + np.array([len(sents[i]) for i in range(len(sents))])
    
    sents_idx = []
    for end in [e.end_char for e in doc.ents]:
        sents_idx.append([i for i, x in enumerate(end <= sents_end) if x][0])
        
    ents_data = []
    for i in range(len(doc.ents)):
        ent = doc.ents[i]
        sent = sents_idx[i]
        if ent.label_ in ['DATE', 'TIME', 'MONEY', 'PERCENT', 'CARDINAL', 'QUANTITY']:
            ents_data.append({
                'value' : ent.text,
                'item' : ent.label_,
                'start' : ent.start_char,
                'end' : ent.end_char,
                'sent' : sent,
                'order' : i
            })
            
    dates = [d for d in ents_data if d['item'] in ['DATE', 'TIME']]
    data = [d for d in ents_data if d['item'] in ['MONEY', 'PERCENT', 'CARDINAL', 'QUANTITY']]
    for v in data:
        v['date'] = get_date(dates, v['sent'], v['order'])
    data = [{'value': d['value'], 'item': d['item'], 'date': d['date']} for d in data]
    data = [list(d.values()) for d in data]
    data.insert(0, ['Value', 'Item', 'Date'])
    output = {
        'value' : data,
        'type' : 'table'
    }
    return(output)

In [7]:
tabular_data_extraction(doc)

{'value': [['Value', 'Item', 'Date'],
  ['20%', 'PERCENT', '2003'],
  ['30%', 'PERCENT', '2004'],
  ['40%', 'PERCENT', '2005']],
 'type': 'table'}